In [1]:
"""
Created a function that compiles multiple noise attributes provided by the Qiskit Aer API.
Different noise models can be created by changing the parameters in the function.

We planned to add more features like ReadoutErrors and others but was unable due to lack of time.
Any future teams are free to add new features as per their requirements.

"""


def create_noise_model(n_qubits):
    
    #Parameters of noise
    # Make these parameters a paramenter to function
    p_bit_flip = {
        'p_reset' : 0.05,
        'p_meas' : 0.15
    }
    
    # !!!Add ReadoutErrors!!!
    u2_error_limits = [3.94e-4, 7.54e-3]
    cx_error_limits = [2e-2, 8e-2]
    
    T_limits = [[11.69e3, 86.12e3], [18.02e3, 132.59e3]] # T2 <= 2*T1    
    instruction_times = {
        #times in nanoseconds
        'time_u1' : 20,
        'time_u2' : 100,
        'time_u3' : 200,
        'time_cx' : 600,
        'time_reset' : 1000,
        'time_measure' : 1000
    }
    #####################################
    # End of paramenter entry
    
    
    T1s = np.random.rand(n_qubits)
    T2s = np.random.rand(n_qubits)
    u2_errors = np.random.rand(n_qubits)
    cx_errors = np.random.rand(n_qubits, n_qubits)
    
    for i in range(n_qubits):
        T1s[i] = T_limits[0][0] + T1s[i]*T_limits[0][1]
        T2s[i] = T_limits[1][0] + T2s[i]*T_limits[1][1]
        u2_errors[i] = u2_error_limits[0] + u2_errors[i]*u2_error_limits[1]
        for j in range(n_qubits):
            cx_errors[i][j] = cx_error_limits[0] + cx_errors[i][j]*cx_error_limits[1]
        
    T2s = np.array([min(T2s[j], 2 * T1s[j]) for j in range(n_qubits)]) # Truncate random T2s <= T1s
    
    #Quantum error objects
    bit_flip_error_reset = pauli_error([('X', p_bit_flip['p_reset']), ('I', 1 - p_bit_flip['p_reset'])])
    bit_flip_error_meas = pauli_error([('X', p_bit_flip['p_meas']), ('I', 1 - p_bit_flip['p_meas'])])
    bit_flip_errors_u2 = [pauli_error([('X', e1), ('I', 1 - e1)]) for e1 in u2_errors]
    
    bit_flip_errors_cx_single = [[pauli_error([('X', e1), ('I', 1-e1)]) for e1 in sub_array]
                                                                            for sub_array in cx_errors]
    bit_flip_errors_cx = [[qe1.tensor(qe1) for qe1 in sub_array] for sub_array in bit_flip_errors_cx_single]
    
    thermal_errors_reset = [thermal_relaxation_error(t1, t2, instruction_times['time_reset'])
                for t1, t2 in zip(T1s, T2s)]
    thermal_errors_measure = [thermal_relaxation_error(t1, t2, instruction_times['time_measure'])
                      for t1, t2 in zip(T1s, T2s)]
    thermal_errors_u1  = [thermal_relaxation_error(t1, t2, instruction_times['time_u1'])
                  for t1, t2 in zip(T1s, T2s)]
    thermal_errors_u2  = [thermal_relaxation_error(t1, t2, instruction_times['time_u2'])
                  for t1, t2 in zip(T1s, T2s)]
    thermal_errors_u3  = [thermal_relaxation_error(t1, t2, instruction_times['time_u3'])
                  for t1, t2 in zip(T1s, T2s)]
    thermal_errors_cx = [[thermal_relaxation_error(t1a, t2a, instruction_times['time_cx']).expand(
                 thermal_relaxation_error(t1b, t2b, instruction_times['time_cx']))
                  for t1a, t2a in zip(T1s, T2s)]
                   for t1b, t2b in zip(T1s, T2s)]
    
    
    
    noise_model = NoiseModel()    
    
    for i in range(n_qubits):
        noise_model.add_quantum_error(bit_flip_error_reset, "reset", [i], warnings = False)
        noise_model.add_quantum_error(bit_flip_error_meas, "measure", [i], warnings = False)
        noise_model.add_quantum_error(thermal_errors_reset[i], "reset", [i], warnings = False)
        noise_model.add_quantum_error(thermal_errors_measure[i], "measure", [i], warnings = False)
        noise_model.add_quantum_error(thermal_errors_u1[i], "u1", [i], warnings = False)
        noise_model.add_quantum_error(thermal_errors_u2[i], "u2", [i], warnings = False)
        noise_model.add_quantum_error(thermal_errors_u3[i], "u3", [i], warnings = False)
        noise_model.add_quantum_error(bit_flip_errors_u2[i], ['u2'], [i], warnings = False)
        for j in range(n_qubits):
            noise_model.add_quantum_error(thermal_errors_cx[i][j], "cx", [i, j], warnings = False)
            noise_model.add_quantum_error(bit_flip_errors_cx[i][j], ['cx'], [i, j], warnings = False)
            
    return noise_model